In [1]:
from langchain_community.document_loaders import PyPDFLoader

# pdf파일 로드
loader = PyPDFLoader("C:\\Users\\user240512\\Desktop\\황신정\\langchain\\pdf\\2018 장애인 차별 예방 사이버인권교육보조교재.pdf")
print(loader.load())

생략

In [2]:
pages = loader.load()

In [3]:
# 일부 페이지만 선택
sel_pages = pages[9:225]

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 문서를 문장으로 분리
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
)
docs = text_splitter.split_documents(sel_pages)

In [5]:
docs[:10]

[Document(metadata={'source': 'C:\\Users\\user240512\\Desktop\\황신정\\langchain\\pdf\\2018 장애인 차별 예방 사이버인권교육보조교재.pdf', 'page': 9}, page_content='차\n이\n･\n차\n별 1강\n이\n야\n기\n제1강 차이･차별 이야기 ◀◀ 3 \n제1강차이･차별 이야기\n차이란 무엇일까요 ? 세상에 나하고 모든 것이 완벽하게 똑같은 사람이 있을까요 ? \n우리는 외형이 매우 비슷한 일란성 쌍둥이를 보면서도 두 사람 사이에 작고 미묘한 차이들이 \n존재함을 느낄 수 있습니다 . 그리고 그렇게 각자에게 존재하는 차이들이 모여 사회 속에서  \n우리를 각각 의미 있는 다른 사람으로 살아가게 합니다 . \n즉, 외모, 성격, 나이, 성적 취향, 학력, 직장, 소득수준 , 출산, 가족형태 , 결혼, 종교, 국적, 피부색 , \n장애 등 우리가 갖고 있는 이러한 촘촘한 차이들을 통해 사회는 획일화되지 않고 다양하게  \n구성되고 운영될 수 있습니다 . \n그렇다면 차별은 무엇일까요 ? 차이와 어떤 관계가 있는 것일까요 ? 누구나 가지고 있는 차이를 \n이유로 다르게 대우하는 것 그것이 바로 차별입니다 . 즉, 장애, 나이, 학력, 성적 취향을 이유로'),
 Document(metadata={'source': 'C:\\Users\\user240512\\Desktop\\황신정\\langchain\\pdf\\2018 장애인 차별 예방 사이버인권교육보조교재.pdf', 'page': 9}, page_content='누군가를 다르게 대우하고 , 누군가에게 불평등한 상황을 초래하는 것을 우리는 차별이라고  \n합니다 . \n따라서 본 차시에서는 차이와 차별에 대한 이해 및 다름이 차별이 되는 상황을 알아보고 , 누구도  \n다름을 이유로 차별 받지 않는 사회환경을 위한 방안을 함께 고민해 보고자 합니다 . \n학습목표\n•차이와 차별, 편견, 선입견 등의 개념을 설명할 수 있다.\n•차이가

In [6]:
import torch
torch.cuda.is_available()

True

In [7]:
import transformers

c:\Users\user240512\.conda\envs\langchain\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from langchain.embeddings import HuggingFaceEmbeddings

# 허깅페이스에 업로드 되어있는 임베딩 모델 선택
embeddings = HuggingFaceEmbeddings(
    model_name='jhgan/ko-sroberta-multitask',
    # model_kwargs={'device':'cpu'},
    model_kwargs={'device':'cuda'},
    encode_kwargs={'normalize_embeddings':True},
)

c:\Users\user240512\.conda\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [31]:
# 벡터 저장소 생성
from langchain.vectorstores import Chroma
import os

vector/store = Chroma.from_documents(docs, embeddings)


# 벡터 저장소 경로 설정
vectorstore_path = 'vectorstore'
os.makedirs(vectorstore_path, exist_ok=True)

# 벡터 저장소 생성 및 저장
vectorstore = Chroma.from_documents(docs, embeddings, persist_directory=vectorstore_path)

# 벡터스토어 데이터를 디스크에 저장
vectorstore.persist()
print("Vectorstore created and persisted")

SyntaxError: invalid syntax (3684965659.py, line 5)

In [28]:
vectorstore._get_retriever_tags

<bound method VectorStore._get_retriever_tags of <langchain_community.vectorstores.chroma.Chroma object at 0x00000277FFF57B20>>

In [11]:
from langchain_community.chat_models import ChatOllama

# Ollama 를 이용해 로컬에서 LLM 실행
model = ChatOllama(model="eeve", temperature=0)

In [12]:
model

ChatOllama(model='eeve', temperature=0.0)

In [20]:
# 유사한 k개의 청크 
retriever = vectorstore.as_retriever(search_type="similarity",search_kwargs={'k': 3})

In [22]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# Prompt 템플릿 생성
template = f'''"친절한 챗봇으로서 상대방의 요청에 최대한 자세하고 친절하게 답하자. 
모든 대답은 한국어(Korean)으로 대답해줘. 
너는 사용자의 입력에 대해 관련된 정보를 정보전달 게시물 형식으로 인스타그램 게시물을 생성해야돼":
{{context}}



Question: {{question}}
'''

prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return '\n\n'.join([d.page_content for d in docs])

# RAG Chain 연결
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)



In [23]:
# Chain 실행
query = "장애 등급 기준"
answer = rag_chain.invoke(query)

print("Query:", query)
print("Answer:", answer)

Query: 장애 등급 기준
Answer: 🌈 장애 등급 기준 이해하기 💡

장애인복지법상 한국에서 인정하는 장애유형은 총 15가지이며, 각 유형별로 판정 기준에 따라 1~6급까지의 등급을 나눠 장애인 지원 제도가 시행되고 있어요. 

🌱 발달장애: 인지적 어려움과 관계 형성에 어려움을 겪는 장애입니다.
🧠 지적장애: 지적 기능 및 적응 기능에 상당한 제한이 있는 장애로, 의사소통과 일상 생활에 어려움이 있습니다.
💭 정신장애: 사고 과정, 감정 조절, 대인관계 등에 문제를 일으키는 장애로, 소통에 어려움을 겪습니다. 

장애 등급은 개인의 능력과 필요를 고려하여 결정되며, 그에 따라 다양한 지원 서비스와 혜택을 받을 수 있어요. 장애인과 비장애인이 함께 어우러져 서로를 이해하고 존중하는 사회를 만들어나가기 위해 노력합시다! #장애등급 #장애인복지법 #발달장애 #지적장애 #정신장애


In [24]:
for chunk in rag_chain.stream("장애 등급 기준"):
    print(chunk, end="", flush=True)

🌈 장애 등급 기준 이해하기 💡

장애인복지법상 한국에서 인정하는 장애유형은 총 15가지이며, 각 유형별로 판정 기준에 따라 1~6급으로 분류됩니다. 이 등급에 따라 장애인 지원 제도가 제공되는데, 1급이 가장 중증이고 6급이 경증입니다.

🌱 발달장애: 인지적 어려움 💡
발달장애는 출생부터 나이가 들면서 정상적으로 발달해야 할 과정에서 어려움을 겪는 장애를 말합니다. 이에는 지적장애와 자폐성장애가 포함됩니다.

🧠 지적장애: 인지 능력 저하 💡
지적장애는 지능이 평균보다 현저히 낮아 일상생활에서 상당한 도움이 필요한 상태를 의미합니다. 이는 학습, 의사소통, 사회적 상호작용에 어려움을 초래합니다.

🌈 자폐성장애: 관계 형성 어려움 💡
자폐성장애는 사회성, 소통, 행동 면에서 심각한 제한을 경험하는 장애입니다. 이로 인해 대인관계 형성과 일상적인 활동에 어려움이 생깁니다.

🧠 정신장애: 의사소통의 어려움 💡
정신장애는 사고 과정, 감정 조절, 사회적 기능에 영향을 미치는 장애로, 의사소통에 어려움을 초래합니다. 이는 조현병이나 양극성 장애와 같은 질환을 포함할 수 있습니다.

🌈 장애 등급과 지원제도 연결하기 💡
장애등급은 장애인 지원 제도의 수준을

KeyboardInterrupt: 

In [25]:
retrieved_docs = retriever.invoke("장애 등급 기준")

len(retrieved_docs)

3

In [26]:
retrieved_docs

[Document(metadata={'page': 35, 'source': 'C:\\Users\\user240512\\Desktop\\황신정\\langchain\\pdf\\2018 장애인 차별 예방 사이버인권교육보조교재.pdf'}, page_content='상황에서 부적절한 정서 반응을 보이는 장애를 뜻한다 . 따라서 지적 장애는 인지, 자폐\n성장애는 관계 맺기, 정신 장애는 소통하는데 어려움을 느끼는 장애이다 .\n나. 장애의 연결\n1. 발달･지적･정신장애의 분류와 증상\n1) 발달･지적･정신장애의 분류\n현재 한국에서는 장애인복지법상 15개 장애유형으로 분류하고 있다.각 장애유형별로 \n판정기준에 따라 1∼6급까지 등급을 나누어 장애인에 대한 지원제도를 등급별로 시행하고 \n있다.\n2) 발달･지적･정신장애의 증상\n(1) 발달장애\n발달장애는 말 그대로 발달에 어려움을 겪는 장애이다 . 사람은 출생부터 나이를 먹어\n가며 발달이 진행되어야 한다. 그러나 어떤 이유로 또래의 같은 연령에 있는 사람과'),
 Document(metadata={'page': 35, 'source': 'C:\\Users\\user240512\\Desktop\\황신정\\langchain\\pdf\\2018 장애인 차별 예방 사이버인권교육보조교재.pdf'}, page_content='상황에서 부적절한 정서 반응을 보이는 장애를 뜻한다 . 따라서 지적 장애는 인지, 자폐\n성장애는 관계 맺기, 정신 장애는 소통하는데 어려움을 느끼는 장애이다 .\n나. 장애의 연결\n1. 발달･지적･정신장애의 분류와 증상\n1) 발달･지적･정신장애의 분류\n현재 한국에서는 장애인복지법상 15개 장애유형으로 분류하고 있다.각 장애유형별로 \n판정기준에 따라 1∼6급까지 등급을 나누어 장애인에 대한 지원제도를 등급별로 시행하고 \n있다.\n2) 발달･지적･정신장애의 증상\n(1) 발달장애\n발달장애는 말 그대로 발달에 어려움을 겪는 장애이다 . 사람은 출생부터 나이를 먹어\n가며 발달이 진행되어야 한다